# DATA PROJECT: A dive into the Danish private sector

**TABLE OF CONTENTS**<a id='toc0_'></a>    
- [1. Project description](#toc1_)    
- [2. Imports and set magics](#toc2_) 
- [3. Read, clean and explore data](#toc3_)
    - 3.1. [Dataset GF02 - active firms given time, industry, size and region](#toc3_1_)
    - 3.2. [Dataset DEMO4 - start-ups and bankruptcies over time and for different industries](#toc3_2_)  
- [4. Merging the datasets](#toc4_)
- [5. Analysis](#toc5_)
- [6. Concluding remarks](#toc6_)
   

## 1. <a id='toc1_'></a>[Project description](#toc0_)

The idea behind the project is to...

## 2. <a id='toc2_'></a>[Imports and set magics](#toc0_)

In [1]:
# As we will use data from Statistics Denmark (Statistikbanken) we will be using the DstApi to access that data.
# If you previously have not installed it through github, the hashtag below should be deleted and the "%pip..." line should be run.
# More info the DstApi can be found by following the link below to the githhub page explaining the Api.

#%pip install git+https://github.com/alemartinello/dstapi


In [2]:
# We import different packages for the further analysis - among these the DstApi that has been installed via the code above.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import ipywidgets as widgets
from matplotlib_venn import venn2
from dstapi import DstApi

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


## 3. <a id='toc3_'></a>[Read, clean and explore data](#toc0_)

### 3.1. <a id='toc3_1_'></a>[Dataset GF02 - active firms given time, industry, size and region](#toc0_)

#### Importing the dataset

In [3]:
gf02_api = DstApi('GF02') 

In [4]:
tabsum = gf02_api.tablesummary(language='en')
display(tabsum)

Table GF02: General enterprise statistics by Municipality, Industry (DB07 10- 19- and 127 grouping and detailed), enterprise size (full time equivalents) and time
Last update: 2023-05-05T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,KOMK,118,000,All Denmark,12,Outside from Denmark,False
1,BRANCHEDB0710TIL127,902,TOT,TOT Industry total,999999,999999 Activity not stated,False
2,FIRMSTR,6,TOT,"Total, all enterprises",103,250 employees and more,False
3,Tid,15,2007,2007,2021,2021,True


In [5]:
# The available values for a each variable: 
for variable in tabsum['variable name']:
    print(variable+':')
    display(gf02_api.variable_levels(variable, language='en'))

KOMK:


,id,text
0,000,All Denmark
1,084,Region Hovedstaden
2,01,Province Byen København
3,101,Copenhagen
4,147,Frederiksberg
...,...,...
113,820,Vesthimmerlands
114,851,Aalborg
115,099,Region unknown
116,098,Municipality unknown


BRANCHEDB0710TIL127:


,id,text
0,TOT,TOT Industry total
1,1,"1 Agriculture, forestry and fishing"
2,A,"A Agriculture, forestry and fishing"
3,01000,01000 Agriculture and horticulture
4,011100,"011100 Growing of cereals (except rice), legum..."
...,...,...
897,990000,990000 Activities of extraterritorial organiza...
898,11,11 Activity not stated
899,X,X Activity not stated
900,99999,99999 Activity not stated


FIRMSTR:


,id,text
0,TOT,"Total, all enterprises"
1,0000,No employed
2,010,Less than 10 employed
3,101,10-49 employees
4,102,50-249 employees
5,103,250 employees and more


Tid:


,id,text
0,2007,2007
1,2008,2008
2,2009,2009
3,2010,2010
4,2011,2011
5,2012,2012
6,2013,2013
7,2014,2014
8,2015,2015
9,2016,2016


In [6]:
params = gf02_api._define_base_params(language='en')
params

{'table': 'gf02',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'KOMK', 'values': ['*']},
  {'code': 'BRANCHEDB0710TIL127', 'values': ['*']},
  {'code': 'FIRMSTR', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [7]:
params_gf02 = {'table': 'gf02',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'KOMK', 'values': ['*']},
  {'code': 'BRANCHEDB0710TIL127', 'values': ['TOT', '1','2','3','4','5','6','7','8','9','10','11']},
  {'code': 'FIRMSTR', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [8]:
gf02_all = gf02_api.get_data(params=params_gf02)
gf02_all.head(5)

,KOMK,BRANCHEDB0710TIL127,FIRMSTR,TID,INDHOLD
0,Brøndby,TOT Industry total,"Total, all enterprises",2007,1588
1,Brøndby,TOT Industry total,No employed,2007,652
2,Brøndby,TOT Industry total,Less than 10 employed,2007,653
3,Brøndby,TOT Industry total,10-49 employees,2007,217
4,Brøndby,TOT Industry total,50-249 employees,2007,51


In [9]:
gf02_all

,KOMK,BRANCHEDB0710TIL127,FIRMSTR,TID,INDHOLD
0,Brøndby,TOT Industry total,"Total, all enterprises",2007,1588
1,Brøndby,TOT Industry total,No employed,2007,652
2,Brøndby,TOT Industry total,Less than 10 employed,2007,653
3,Brøndby,TOT Industry total,10-49 employees,2007,217
4,Brøndby,TOT Industry total,50-249 employees,2007,51
...,...,...,...,...,...
97876,Skanderborg,8 Other business services,"Total, all enterprises",2013,497
97877,Skanderborg,8 Other business services,No employed,2013,302
97878,Skanderborg,8 Other business services,Less than 10 employed,2013,174
97879,Skanderborg,8 Other business services,10-49 employees,2013,18


#### Cleaning and renaming the dataset

In [10]:
gf02_all.rename(columns = {'KOMK':'regions', 'BRANCHEDB0710TIL127':'industry', 'FIRMSTR': 'firm size', 'TID':'year', 'INDHOLD':'#firms'}, inplace=True)
gf02_all.head()

,regions,industry,firm size,year,#firms
0,Brøndby,TOT Industry total,"Total, all enterprises",2007,1588
1,Brøndby,TOT Industry total,No employed,2007,652
2,Brøndby,TOT Industry total,Less than 10 employed,2007,653
3,Brøndby,TOT Industry total,10-49 employees,2007,217
4,Brøndby,TOT Industry total,50-249 employees,2007,51


In [ ]:
#Call function from py-file
#var_keep = ['region', 'All Denmark']
#gf02_filtered = dataproject.keep_regs(gf02_all, var_keep)

In [11]:
I = gf02_all.regions.str.contains('Region')
I |= gf02_all.regions.str.contains('All Denmark')
gf02_all.loc[I, :]

,regions,industry,firm size,year,#firms
152,All Denmark,"9 Public administration, education and health","Total, all enterprises",2012,24676
153,All Denmark,"9 Public administration, education and health",No employed,2012,11561
154,All Denmark,"9 Public administration, education and health",Less than 10 employed,2012,10269
155,All Denmark,"9 Public administration, education and health",10-49 employees,2012,2148
156,All Denmark,"9 Public administration, education and health",50-249 employees,2012,488
...,...,...,...,...,...
97344,Region Hovedstaden,3 Construction,No employed,2017,4171
97345,Region Hovedstaden,3 Construction,Less than 10 employed,2017,4614
97346,Region Hovedstaden,3 Construction,10-49 employees,2017,634
97347,Region Hovedstaden,3 Construction,50-249 employees,2017,83


In [12]:
gf02_reduce = gf02_all.loc[I == True] # drop everything else
gf02_reduce.reset_index(inplace = True, drop = True) # Drop old index too. If the drop is not used, python will create another dataset with the old index.
gf02_reduce.iloc[0:153,:]
gf02_reduce

,regions,industry,firm size,year,#firms
0,All Denmark,"9 Public administration, education and health","Total, all enterprises",2012,24676
1,All Denmark,"9 Public administration, education and health",No employed,2012,11561
2,All Denmark,"9 Public administration, education and health",Less than 10 employed,2012,10269
3,All Denmark,"9 Public administration, education and health",10-49 employees,2012,2148
4,All Denmark,"9 Public administration, education and health",50-249 employees,2012,488
...,...,...,...,...,...
6093,Region Hovedstaden,3 Construction,No employed,2017,4171
6094,Region Hovedstaden,3 Construction,Less than 10 employed,2017,4614
6095,Region Hovedstaden,3 Construction,10-49 employees,2017,634
6096,Region Hovedstaden,3 Construction,50-249 employees,2017,83


In [14]:
gf02_reduce.describe()

,year,#firms
count,6098.000000,6098.000000
mean,2013.972614,6060.846179
std,4.330992,20753.690834
min,2007.000000,1.000000
25%,2010.000000,60.000000
50%,2014.000000,888.000000
75%,2018.000000,4157.500000
max,2021.000000,328445.000000


#### Exploring the dataset

In [15]:
# Data frame
test1 = gf02_reduce.loc[gf02_reduce['regions'] == 'Region Hovedstaden',
                        gf02_reduce['industry'] == 'TOT Industry total',
                        gf02_reduce['firm size'] == 'Total, all enterprises']

# Plot the content of the data frame
test1.plot(x='year',y='#firms',legend=False);

IndexingError: Too many indexers

### 3.2. <a id='toc3_2_'></a>[Dataset DEMO4 - start-ups and bankruptcies over time and for different industries](#toc0_)

#### Importing the dataset

As we are using another dataset from Statistics Denmark, we apply the same approach as described with the previous dataset (GF02).

In [ ]:
ind2 = DstApi('DEMO4')

In [ ]:
tabsum2 = ind2.tablesummary(language='en')
display(tabsum2)

In [ ]:
# The available values for a each variable: 
for variable in tabsum2['variable name']:
    print(variable+':')
    display(ind2.variable_levels(variable, language='en'))

In [ ]:
params = ind2._define_base_params(language='en')
params

In [ ]:
params2 = {'table': 'demo4',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'REGION', 'values': ['*']},
  {'code': 'BRANCHEDB0710', 'values': ['*']},
  {'code': 'MÆNGDE4', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [ ]:
inc_api2 = ind2.get_data(params=params2)
inc_api2.head(5)

#### Cleaning and renaming the dataset

In [ ]:
inc_api2.rename(columns = {'BRANCHEDB0710':'Industry', 'MÆNGDE4': 'X', 'TID':'Time', 'INDHOLD':'Total'}, inplace=True)
inc_api2.head()

#### Exploring the dataset

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

## 4. <a id='toc4_'></a>[Merging the datasets](#toc0_)

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

## 5. <a id='toc5_'></a>[Analysis](#toc0_)

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

## 6. <a id='toc6_'></a>[Concluding remarks](#toc0_)

ADD CONCISE CONLUSION.